In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering, KMeans
from numpy.linalg import svd
from scipy.linalg import sqrtm
from scipy.sparse import csgraph
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split

In [ ]:
user_ratings = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment2/train.csv')
user_ratings.head()

In [ ]:
test = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment2/test.csv', header=None, names=['movie_id','customer_id','rating','date'])

In [ ]:
movie_titles = pd.read_csv(r'/Users/niharika/Documents/Niharika/Third Quarter/ECS 271 ML/Assignment2/movie_titles.txt', sep=',', header=None, encoding="ISO-8859-1", names=["movie_id","date_produced","title"])
movie_titles = movie_titles.iloc[1: , :]
movie_titles = movie_titles.fillna('')
movie_titles.head()
# temp = pd.DataFrame(movie_titles['title1'] + movie_titles['test1'] + movie_titles['test2'])
# temp.rename( columns={0:'title1'}, inplace=True )
# temp

In [ ]:
# movie_titles1 = movie_titles.join(temp)
# movie_titles1

In [ ]:
# mv_df = pd.DataFrame(movie_titles['movie_id'].astype(int))
# mv_df

In [ ]:
mv_df = pd.DataFrame(movie_titles['movie_id'].astype(int).unique(),columns=['movie_id'])
mv_df

In [ ]:
cus_df = pd.DataFrame(user_ratings.customer_id.astype(int).unique(), columns=['customer_id'])
cus_df

In [ ]:
train, test1 = train_test_split(user_ratings, test_size = 0.2)
test1.shape

In [ ]:
master = pd.merge(cus_df, mv_df, how = 'cross')
master

In [ ]:
new_master = pd.merge(master,user_ratings,on=["movie_id","customer_id"],how='left')
new_master = new_master.drop_duplicates()
new_master

In [ ]:
cmr_matrix_1 = new_master[['customer_id','movie_id','rating']]

In [ ]:
del mv_df, cus_df, master, new_master, movie_titles, user_ratings

In [ ]:
cmr_matrix = cmr_matrix_1.pivot(index = 'customer_id', columns ='movie_id', values = 'rating')
cmr_matrix

In [ ]:
users = list(cmr_matrix.index)
users_index = {users[i]: i for i in range(len(users))}
itemcols = list(cmr_matrix.columns)
movie_index = {itemcols[i]: i for i in range(len(itemcols))}
print(users_index, movie_index) 

In [ ]:
uMatrix = np.array(cmr_matrix)
mask = np.isnan(uMatrix)
masked = np.ma.masked_array(uMatrix, mask)
item_means = np.mean(masked, axis=0)

In [ ]:
uMatrix = masked.filled(item_means)
uMatrix

In [ ]:
x = np.tile(item_means, (uMatrix.shape[0],1))
uMatrix = uMatrix - x

In [ ]:
u, s, vt = svd(uMatrix, full_matrices = False)

In [ ]:
print(s.shape,u.shape,vt.shape)

In [ ]:
s = np.diag(s)

In [ ]:
s=s[0:10,0:10]
u=u[:,0:10]
vt=vt[0:10,:]

In [ ]:
sRoot=sqrtm(s)

In [ ]:
Usk=np.dot(u,sRoot)
skV=np.dot(sRoot,vt)
UsV = np.dot(Usk, skV)
UsV = UsV + x
UsV

In [ ]:
del uMatrix, mask, masked, item_means, Usk, skV, s, u, vt, x, sRoot

In [ ]:
for user, _ in cmr_matrix.iterrows():
    for movie in itemcols:  
        if user in users_index:        
            u_index = users_index[user]
            if movie in movie_index:
                m_index = movie_index[movie]
                pred_rating = UsV[u_index, m_index]
            else:
                print("Movie not found")
                pred_rating = np.mean(UsV[u_index, :])
        else:
            print("User not found")
            pred_rating = np.mean(UsV[:, m_index])
        cmr_matrix.at[user,movie] = pred_rating
print(cmr_matrix)

In [ ]:
cmr_matrix = cmr_matrix.apply(lambda row: row.fillna(row.mean()), axis=1)

In [ ]:
pred = []

for _,row in test1.iterrows():
    user = row['customer_id']
    movie = row['movie_id']
    if user in users_index:
        u_index = users_index[user]
        if movie in movie_index:
            m_index = movie_index[movie]
            pred_rating = UsV[u_index, m_index]
        else:
            pred_rating = np.mean(UsV[u_index, :])

    else:
        pred_rating = np.mean(UsV[:, m_index])        
    pred.append(pred_rating)

print(mse(test1['rating'], pred))

In [ ]:
cmr = np.array(cmr_matrix)
X = cosine_similarity(cmr)
# np.save(r'Documents/user_pred_matrix.npy',cmr)

In [ ]:
def eigenDecomposition(A, plot = True, topK = 5):

    L = csgraph.laplacian(A, normed=True)
    n_components = A.shape[0]
    
    # LM parameter : Eigenvalues with largest magnitude (eigs, eigsh), that is, largest eigenvalues in 
    # the euclidean norm of complex numbers.
#     eigenvalues, eigenvectors = eigsh(L, k=n_components, which="LM", sigma=1.0, maxiter=5000)
    eigenvalues, eigenvectors = np.linalg.eig(L)
        
    # Identify the optimal number of clusters as the index corresponding
    # to the larger gap between eigen values
    index_largest_gap = np.argsort(np.diff(eigenvalues))[::-1][:topK]
    nb_clusters = index_largest_gap + 1
        
    return nb_clusters, eigenvalues, eigenvectors

In [ ]:
k, evl,evc = eigenDecomposition(X)
print(f'Optimal number of clusters {k}')

In [ ]:
n = 100

sc=SpectralClustering(affinity='precomputed', n_clusters=n, n_components=10,random_state=25).fit(X)
labels = sc.labels_
labels
# s1 = np.array(['#B7A4DA','#BFF5E5','#A6E1EA','#FCB8B8','#F2ECCE','#CC859A','#9EAEB2','#62A6BF','black','red','green'])
# plt.scatter(cmr[:,0], cmr[:,1], c=labels,s=1)

In [ ]:
np.save(r'Documents/labels.npy',labels)

In [ ]:
labels = np.load(r'/Users/niharika/Downloads/idec_pred6.npy')

In [ ]:
uid_to_cluster = { key:value for key,value in zip( cmr_matrix.index, labels ) }
cluster_to_uids = [[] for x in range(100)]
for uid, cluster_no in uid_to_cluster.items():
    cluster_to_uids[cluster_no].append(uid)

In [ ]:
def estimate_rating(uid, mid):
    temp = []
    try:
        uids = cluster_to_uids[uid_to_cluster[uid]]
        u_indices = [users_index[x] for x in uids]
    except KeyError:
        return 3

    try:
        m_index = movie_index[mid]
    except KeyError:
        return 3
    
    for i in u_indices:
        temp.append(cmr[i,m_index])
#     print(temp,m_index)
    average_rating = round(np.mean(temp))
    return average_rating

def mse(true, pred):
    x = true - pred
    return sum([xi*xi for xi in x])/len(x)

In [ ]:
pred = []
file = open(r'/Users/niharika/Documents/ratings_sc.txt','w')
total = 0
correct = 0
for _,row in test.iterrows():
    user = row['customer_id']
    movie = row['movie_id']
    pred_rating = estimate_rating(user,movie)
    if pred_rating > 5:
        pred_rating = 5
    elif pred_rating < 1:
        pred_rating = 1
#     pred.append(pred_rating)   
#     total += 1
#     if pred_rating == row['rating']:
#         correct += 1
    file.write(str(pred_rating))
    file.write('\n')
# print(pred)
file.close()
# print(mse(test1['rating'], pred), total, correct)